### Supperstore Sales Forecasting with SARIMA
#### 1. Introduction
##### This project focuses on implementing SARIMA to forecast monthly sales for Superstore.
##### The objectives are:
- ##### To analyze historical sales trends
- ##### To remove trend and seasonality for better forecasting
- ##### To generate accurate 12-month forecasts using both generic and rolling methods.
- ##### To evaluate the model(s) performance using RMSE and MAPE

In [ ]:
# Getting necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

#### 2. Data Preparation & Trend Analysis
##### The dataset contains monthly sales from 01-Jan-2011 to 31-Dec-2014
##### Key Trend Observations:
- ##### Sales exhibit strong seasonality with recurring dips and peaks in the 1st and 4th quarters respectively for the three years
- ##### A general upward trend is observed in the dataset.
##### This justifies the use of seasonal ARIMA

In [ ]:
# Loading the dataset
df = pd.read_excel("superstore_sales.xlsx", parse_dates=["order_date"])
df.set_index("order_date", inplace=True)

# Aggregating sales data to monthly level
monthly_sales = df["sales"].resample("M").sum()

# Plotting the sales trend
fig = go.Figure()
fig.add_trace(go.Scatter(x=monthly_sales.index, y=monthly_sales, mode='lines', name='Monthly Sales'))
fig.update_layout(title='Monthly Sales Trend', xaxis_title='Date', yaxis_title='Sales')
pio.show(fig)

#### 3. Stationarity Check & Differencing


In [ ]:
# ADF Test for stationarity
def adf_test(series):
    result = adfuller(series)
    print("ADF Statistic:", result[0])
    print("p-value:", result[1])
    if result[1] > 0.05:
        print("Data is not stationary. Differencing is needed.")
    else:
        print("Data is stationary.")

adf_test(monthly_sales)

##### ADF Test Results: The original time series was not stationary (p-value = 0.988)
##### I therefore applied First-order differencing followed by seasonal differencing (lag=12), achieving stationarity (p-value = 3.708)

In [ ]:
# First-order differencing
diff_sales = monthly_sales.diff().dropna()
adf_test(diff_sales)

# Seasonal differencing (period=12)
seasonal_diff_sales = diff_sales.diff(12).dropna()
adf_test(seasonal_diff_sales)


##### Performing Time Series Decomposition to verify the presence of both trend and seasonal components, reinforcing the need for differencing.

In [ ]:
# Decomposition
result = seasonal_decompose(monthly_sales, model='additive', period=12)
fig = go.Figure()
fig.add_trace(go.Scatter(x=monthly_sales.index, y=result.trend, mode='lines', name='Trend'))
fig.update_layout(title='Trend Component', xaxis_title='Date', yaxis_title='Sales')
pio.show(fig)

fig = go.Figure()
fig.add_trace(go.Scatter(x=monthly_sales.index, y=result.seasonal, mode='lines', name='Seasonality'))
fig.update_layout(title='Seasonality Component', xaxis_title='Date', yaxis_title='Sales')
pio.show(fig)


#### 4. SARIMA Model Selection & Performance

In [ ]:
# Train-test split
train = monthly_sales[:-12]
test = monthly_sales[-12:]

# Auto-SARIMA model selection
sarima_model = auto_arima(train, seasonal=True, m=12, stepwise=True, trace=True)
best_order, best_seasonal_order = sarima_model.order, sarima_model.seasonal_order
print(f"Optimal SARIMA Order: {best_order}, Seasonal Order: {best_seasonal_order}")

##### The optimal model is SARIMA(0,0,0)(2,1,0,12) identified via auto_arima.
##### I used this values to train the SARIMA model as follows:

In [ ]:
# Train SARIMA model with optimized parameters
sarima = SARIMAX(train, order=(0, 0, 0), seasonal_order=(2, 1, 0, 12))
sarima_fit = sarima.fit()

#### 5. Sales Forecast Results
##### Generic Forecast (One-time 12-month prediction)
##### The model forecasts the next 12 months based on historical patterns.

In [ ]:
# Generic/Fixed Forecasting
forecast = sarima_fit.get_forecast(steps=12)
forecast_ci = forecast.conf_int()

fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train, mode='lines', name='Actual Sales'))
fig.add_trace(go.Scatter(x=test.index, y=test, mode='lines', name='Test Data', line=dict(dash='dot')))
fig.add_trace(go.Scatter(x=forecast.predicted_mean.index, y=forecast.predicted_mean, mode='lines', name='SARIMA Forecast', line=dict(color='red')))
fig.update_layout(title='12-Month SARIMA Forecast', xaxis_title='Date', yaxis_title='Sales')
pio.show(fig)

#### Rolling Forecast (Step-by-step prediction)
#### Rolling forecasting updates predictions iteratively, adapting to recent sales trends.

In [ ]:
# Rolling Forecasting - Using Best SARIMA Order
rolling_predictions = []
history = list(train)

for i in range(len(test)):
    model = SARIMAX(history, order=(0, 0, 0), seasonal_order=(2, 1, 0, 12))
    model_fit = model.fit()
    pred = model_fit.forecast()[0]
    rolling_predictions.append(pred)
    history.append(test.iloc[i])
rolling_predictions = pd.Series(rolling_predictions, index=test.index)

fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train, mode='lines', name='Actual Sales'))
fig.add_trace(go.Scatter(x=test.index, y=test, mode='lines', name='Test Data', line=dict(dash='dot')))
fig.add_trace(go.Scatter(x=rolling_predictions.index, y=rolling_predictions, mode='lines', name='Rolling SARIMA Forecast', line=dict(color='green')))
fig.update_layout(title='Rolling SARIMA Forecast', xaxis_title='Date', yaxis_title='Sales')
pio.show(fig)

#### 6. Model Evaluation

In [ ]:
# Calculating RMSE and MAPE
def evaluate_forecast(actual, predicted, model_name):
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    print(f"🔹 {model_name} - RMSE: {rmse:.2f}, MAPE: {mape:.2f}%")
    return rmse, mape

# Evaluating Generic SARIMA Forecast
generic_rmse, generic_mape = evaluate_forecast(test, forecast.predicted_mean, "Generic SARIMA")

# Evaluating Rolling SARIMA Forecast
rolling_rmse, rolling_mape = evaluate_forecast(test, rolling_predictions, "Rolling SARIMA")

# Summary Table
evaluation_results = pd.DataFrame({
    "Model": ["Generic SARIMA", "Rolling SARIMA"],
    "RMSE": [generic_rmse, rolling_rmse],
    "MAPE (%)": [generic_mape, rolling_mape]
})

print("\nModel Evaluation Summary:")
print(evaluation_results)


##### The Generic SARIMA and Rolling SARIMA models produced very similar results:
- ##### Generic SARIMA: RMSE = 79,301.46, MAPE = 15.47%
- ##### Rolling SARIMA: RMSE = 79,209.05, MAPE = 15.62%
##### Interpretation of Results
- ##### Low RMSE values → Both models provide similar forecasting accuracy.
- ##### MAPE around 15% → Forecasts have a moderate error rate (considered reasonable for business forecasting).
- ##### Rolling SARIMA has slightly lower RMSE → Rolling updates may provide minor improvements, but the difference is minimal.


#### 7. Business Insights & Recommendations
##### Peak Sales Periods: Higher sales expected in the 4th quarter. Superstore should increase inventory and marketing efforts.
##### Low Sales Periods: Lower sales are expected in the first quarter. Promotions and discounts can help maintain revenue during these months.
##### Model Enhancements: Future iterations could incorporate external factors like promotions, economic trends, or weather impacts.